## GUI

Este proyecto utiliza Gradio para crear una interfaz de chat que interactúa con el modelo Llama3.2, permitiendo al usuario hacer preguntas sobre diversos temas. Las respuestas se generan a partir de textos de ejemplo proporcionados, lo que facilita la interacción y consulta directa con el modelo

Importa las librerías necesarias

In [18]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
import gradio as gr
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

Carga los pdfs, extraemos los textos y los dividimos

In [7]:
pdf_files = ["documents/pruebaLLM.pdf","documents/pruebaLLM2.pdf", "documents/MartinScorsese.pdf"]  

all_documents = []

for document in pdf_files:
    loader = PyPDFLoader(document) # Carga el documento
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)    # Divide el texto en chunks
    documents = text_splitter.split_documents(data)
    all_documents.extend(documents)

print(f"Se cargaron y dividieron {len(all_documents)} fragmentos de texto.")

Se cargaron y dividieron 220 fragmentos de texto.


Configura un modelo de Hugging Face para generar embeddings de texto.

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Crear un vector store de Chroma usando los documentos y el modelo de embeddings


In [9]:
vector_store = Chroma.from_documents(
    documents=all_documents,
    collection_name="pdf-prueba6",
    embedding=embeddings,
)

Utiliza una función de embeddings para representar los datos

In [10]:
vector_store = Chroma(
    collection_name="pdf-prueba6",
    embedding_function=embeddings,
)

Convierte el vector store en un retriever  para realizar consultas basadas en similitud de texto

In [11]:
retriever = vector_store.as_retriever()

Pregunta de ejemplo

In [12]:
question = "Como me llamo?"
docs = vector_store.similarity_search(question,k=10)
len(docs)

10

Vemos los textos más relacionados a la pregunta

In [13]:
docs

[Document(metadata={'page': 13, 'source': 'documents/MartinScorsese.pdf'}, page_content="todo el lirismo, la sutileza y la potencia que se le encomienda. Alo que es preciso añadir los\natractivosrasgospropiosdelafisonomíaderaízitaliana,queScorseserequeríaparainterpretar\nelpersonaje de TeresaRonchelli.\nEN ELNIGHT CLUB\nElVolpe's, sugiere, entérminosonomásticos, laideadeguarida. Elrefugiodelanimal.\nRegentadoporTony, elVolpe's es lugardeencuentro de ítalo-norteamericanos. Espartede los"),
 Document(metadata={'page': 16, 'source': 'documents/MartinScorsese.pdf'}, page_content='las arenas de unaplaya dorada, desierta, otoñal. Esto es, la luz otorganuevamente a lapareja\nuna tonalidad onírica. Es la luminosidad del amor, que se asocia a Teresa. Supostura es más\nlúcida que la de Charlie, quien se compara con San Francisco en su compromiso deproteger.\nTeresa es en ese momento, un personaje más íntegro, o maduro, que Charlie. Sus apariciones'),
 Document(metadata={'page': 4, 'source': 'doc

Crea una cadena de procesamiento de lenguaje natural utilizando un LLM local (Ollama) para responder preguntas basadas en un contexto dado

In [14]:
# Prompt: Se define una plantilla de pregunta-respuesta que toma un contexto y una pregunta como entrada
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM: Se configura el modelo llama3.2 para generar respuestas de manera local
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain Se construye un pipeline que pasa el contexto desde el retriever
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

Consulta la base de datos de vectores y devuelve una respuesta utilizando un modelo LLM

In [15]:
def search_chroma(input,history):
    try:
        # Invocar la cadena con la consulta como input
        response = chain.invoke(input)
        return response  
    except Exception as e:
        return f"Error al procesar la consulta: {e}"

Crea una interfaz de chat con Gradio que permite interactuar con el modelo para responder preguntas basadas en textos

In [16]:
# Define la interfaz de chat
with gr.Blocks(theme=gr.themes.Glass()) as demo:
    gr.Markdown("### Chat con llama3.2")
    
    # Crea el ChatInterface para manejar la interacción
    chat_interface = gr.ChatInterface(fn=search_chroma,type="messages", # The default value (type="tuples") is deprecated 
                                       examples=["Quien es Martin Scorsese", "Que relación tiene la violencia es sus películas?", "Háblame de Goodfellas"],
                                     )

# Lanza la interfaz
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


En conclusión, la interfaz de chat con Llama3.2 proporciona una manera sencilla y efectiva de obtener respuestas a preguntas sobre una variedad de temas, utilizando un modelo de lenguaje potente y accesible a través de Gradio. Esto facilita la interacción con el modelo y mejora la experiencia del usuario.